#### 유튜브 댓글 크롤링 해오기

In [204]:
# 필요한 라이브러리 임포트
from selenium import webdriver
import selenium
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')


[1] 크로릴할 유튜브 불러오기
***

In [205]:
# 유튜브 영상 URL
video_url = 'https://www.youtube.com/watch?v=mgUrWDxf7oo'  # 유튜브 영상 URL

# 유튜브 페이지 열기
def open_youtube_page(video_url):
    service = Service(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    driver.get(video_url)
    driver.implicitly_wait(3)
    return driver

driver = open_youtube_page(video_url)  # 유튜브 페이지 열기


[3] 웹페이지 끝까지 스크롤하기
***

In [206]:
# 웹페이지 스크롤 하기
def scroll_to_bottom(driver):
    last_height = driver.execute_script('return document.documentElement.scrollHeight')

    while True:
        driver.execute_script('window.scrollTo(0, document.documentElement.scrollHeight);')
        time.sleep(1.5)
        
        new_height = driver.execute_script('return document.documentElement.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height
    time.sleep(1.5)

scroll_to_bottom(driver)  # 페이지 스크롤


In [207]:
# 대댓글 버튼 클릭
def click_more_replies(driver):
    buttons = driver.find_elements(By.CSS_SELECTOR, '#more-replies > a')
    time.sleep(1.5)

    for button in buttons:
        button.send_keys(Keys.ENTER)
        time.sleep(1.5)
        button.click()

click_more_replies(driver)  # 대댓글 클릭


In [208]:
# 댓글 가져오기
def get_comments(driver):
    # 현재 페이지의 HTML 소스를 가져옴
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')  # BeautifulSoup으로 HTML 파싱

    # 댓글 내용을 선택
    comment_list = soup.select('yt-attributed-string#content-text')
    comment_final = []  # 댓글을 저장할 리스트 초기화

    # 각 댓글을 반복하면서 내용 추출
    for comment in comment_list:
        temp_comment = comment.text  # 댓글 텍스트 가져오기
        temp_comment = temp_comment.replace('\n', '').replace('\t', '').strip()  # 불필요한 공백 및 줄바꿈 제거
        comment_final.append(temp_comment)  # 리스트에 추가
    
    return comment_final  # 최종 댓글 리스트 반환

comments = get_comments(driver)  # 댓글 가져오기


In [209]:
# 댓글을 엑셀로 저장하기
def save_comments_to_excel(comments):
    pd_data = {'댓글 내용': comments}
    youtube_pd = pd.DataFrame(pd_data)
    youtube_pd.to_excel('result20.xlsx', index=False)
    print("댓글이 result.xlsx에 저장되었습니다.")

save_comments_to_excel(comments)  # 댓글 저장


댓글이 result.xlsx에 저장되었습니다.


In [210]:
driver.quit()  # 드라이버 종료
